# Truss Analyer Testing Zone

In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import numpy as np
import pandas as pd
import re
import math
from PyNite import FEModel3D
from PyNite.Visualization import render_model
from rich import print
import pandas as pd
import matplotlib.pyplot as plt
# import truss_input as ti
import truss_model as tm

In [97]:
# Import Canam's tables

def remove_comma(cell):
    number = float(cell.replace(',', ''))
    return number

canam_df = pd.read_csv('OWSJ_tables_canam.csv',header=1)
canam_df.rename(columns={canam_df.columns[0]: 'span', canam_df.columns[1]: 'depth'}, inplace=True)
canam_df['span'].ffill(inplace=True)
canam_df['span']= canam_df['span']*1000
canam_df['depth'] = canam_df['depth'].apply(remove_comma)
canam_df['depth'] = canam_df['depth'].astype(float)
canam_df.head(10)

# save as json for linux reading
canam_df.to_json('canam_df.json', orient='records')

In [98]:
# Import Vulcraft's data
vulcraft_df = pd.read_csv('OWSJ_tables_vulcraft.csv', header=1)
vulcraft_df.rename(columns={"Factored Load": 'value'}, inplace=True)
vulcraft_df.columns = [col.lower() for col in vulcraft_df]
#drop first row and all bridging rows
vulcraft_df.drop(index=0, inplace=True)
vulcraft_df = vulcraft_df[~vulcraft_df['value'].str.contains('Bridg.')]
#fill depths
vulcraft_df['depth'].bfill(inplace=True)
#convert all span values to mm
vulcraft_df.span = vulcraft_df.span.astype(float)
vulcraft_df.span = vulcraft_df.span*1000
vulcraft_df.head(30)

# save as json for linux reading
vulcraft_df.to_json('vulcraft_df.json', orient='records')

In [99]:
# Import Omega's tables

def find_span(cell):
    if "SPAN" in cell:
        match_span = re.search(r'\b(\d+)\b', cell)
        return match_span.group(1)
    else:
        return np.nan

omega_df = pd.read_csv('OWSJ_tables_omega.csv', header=None)
col_names = ["depth", "D/D", "E/D", "E/E", "F/E", "F/F", "G/F", "G/G", "H/G", "H/H", "K/H", "K/K", "L/K", "L/L"]
omega_df.columns = col_names
omega_df['span'] = omega_df['depth'].apply(find_span)
omega_df['span'].ffill(inplace=True)
columns = ['span'] + [col for col in omega_df if col != 'span']
omega_df = omega_df[columns]
# Drop rows with useless info; cells with "DEPTH" OR "SPAN"
omega_df = omega_df[~omega_df['depth'].str.contains('DEPTH') & ~omega_df['depth'].str.contains('SPAN')]

# save as json for linux reading
omega_df.to_json('omega_df.json', orient='records')

In [43]:
import tabula
tables = tabula.read_pdf("metric tables from Vulcraft OWSJ.pdf", pages='all', multiple_tables=True, pandas_options={'header': None})


In [ ]:
top_nodes = []
bot_nodes = []
d_w = L/(n/2)
for i, loc in enumerate(list(np.linspace(0, L, int(n/2+1)))):
    t_node = [loc, d]
    top_nodes.append(t_node)
    if i == 0:
        continue
    else: 
        b_node = [loc-d_w/2, 0] 
        bot_nodes.append(b_node)

In [85]:
vulcraft_df.depth.dtype

dtype('O')